`updates from the last main_model_code.ipynb in master branch.`

`0.1 possibly some new libraries`

`2.1 I moved this code chunk from the bottom to here.`

`2.3 New code chunk -- this gets the Y label for the decoder`

`2.6 Major code change here to allow encoder option`

`3.2 New code chunk -- this calculates cosine similarity loss for decoder.`

`3.4, 3.5 compile_model() function is now customized as two compile_caps_model() and compile_cnn_model() for code claritys sake. corresponding loss functions and loss weights are updated`

`(you can delete the two existing code chunks which contains `def decoder_loss()` and compile_model())`

`3.6 fit_model() arguments have changed to allow for multiple model outputs, replace this whole chunk`

`4.0 'embed_dim' : embed_dim;  swtich 'use_decoder' to True to enable decoder`

`4.1 'embed_dim' : embed_dim`

`4.2 compile_model() now becomes compile_caps_model`

`4.3 compile_model() now becomes compile_cnn_model`

`4.4 New code chunk -- draw and compile CapsNet with decoder`

`4.5, 4.6 minor argument format change according to change described in 3.6`

`4.7 New code chunk to fit CapsNet with decoder`

### 0.0 Load Libraries

In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import tensorflow as tf
import keras as K
from keras import callbacks, optimizers
from keras import backend as KB
from keras.engine import Layer
from keras.layers import Activation
from keras.layers import LeakyReLU, Dense, Input, Embedding, Dropout, Reshape, Concatenate, MaxPooling1D, Flatten
from keras.layers import Bidirectional, GRU, Flatten, SpatialDropout1D, Conv1D
from keras.layers import Add
# from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical
from common import vocabulary, utils

# capsule layers from Xifeng Guo 
# https://github.com/XifengGuo/CapsNet-Keras
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
import glove_helper

import time # !

/home/yeunghoman/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [87]:
from evaluation_helper import EvalDev_Report

### 0.1 

In [2]:
print("Tensorflow version:", tf.__version__)
print("Keras version:", K.__version__)

Tensorflow version: 1.4.1
Keras version: 2.1.5


### 0.2 

In [3]:
TRAIN_FILE = "../data/conll2003/eng.train"
DEV_FILE = "../data/conll2003/eng.testa"
TEST_FILE = "../data/conll2003/eng.testb"

## Data Class !

### 1.0 

In [4]:
# local untils

# timeit decorator
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

### 1.1

In [5]:
def construct_embedding_matrix(embed_dim):
    """
    construct embedding matrix from GloVe 6Bn word data
    
    reuse glove_helper code from w266 
    
    Returns: an embedding matrix directly plugged into keras.layers.Embedding(weights=[embedding_matrix])
    """
    reload(glove_helper)
    hands = glove_helper.Hands(ndim=embed_dim)
    embedding_matrix = np.zeros((vocabData.vocab.size, embed_dim))

    for i in range(vocabData.vocab.size):
        word = vocabData.vocab.ids_to_words([i])[0]
        try:
            embedding_vector = hands.get_vector(word)
        except:
            embedding_vector = hands.get_vector("<unk>")
        embedding_matrix[i] = embedding_vector

    return embedding_matrix

### 1.2

In [6]:
class conll2003Data(object):
    """
    Keep track of data and processing operations for a single CoNLL2003 data file.
    """

    def __init__(self, filePath_train):
        """
        filePath(string): path to a CoNLL2003 raw data file for training the vocabulary
        """
        self.vocab = []
        self.posTags = []
        self.nerTags = []
        self.train_sentences = self.readFile(filePath_train)


    @timeit
    def readFile(self, filePath, canonicalize=True, verbose=False):
        """
        Read the conll2003 raw data file

        filename(string) - path to conll2003 file (train, test, etc.)
        
        Returns: a list of lists of lists corresponding to the words, pos tags, ner tags, capitalization
                 in each sentence

        """
        print ("----------------------------------------------------")
        print ("reading file from path", str(filePath))
        f = open(filePath)
        sentences = []
        sentence = []
        for line in f:
            if len(line) == 0 or line.startswith("-DOCSTART") or line[0] == '\n':
                if len(sentence) > 0:
                    sentences.append(sentence)
                    sentence = []
                continue
            
            # input format is [ word, pos tag, chunck tag, ner tag]
            # we are ignoring the chunck tag
            splits = line.strip().split(' ')
            if canonicalize:
                word = [utils.canonicalize_word(splits[0]), splits[1], splits[3], self.capitalizaion(splits[0])]
            else:
                word = [splits[0], splits[1], splits[3], capitalizaion(splits[0])]
            sentence.append( word)
        
        # don't forget the last sentence
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence = []
        
        if verbose: 
            print ("number of sentences on file =",len(sentences))
            print ("first 5 sentences:")
            print (sentences[:5])

        return sentences
    
    
    def capitalizaion(self, word):
        """
        check capitalization info for a word
        return 'lowercase' for 'sfsd'
        return 'allCaps' for 'SFSD'
        return 'upperInitial' for 'Sfsd'
        return 'mixedCaps' for 'SfSd'
        return 'noinfo' for '$#%@#' or '12334'
        """
        alphas = [c.isalpha() for c in word] 
        if sum(alphas) != len(word):
            return 'noinfo'
        caps = [char.lower()==char for char in word]
        if sum(caps) == len(word):
            return 'lowercase'
        elif sum(caps) == 0:
            return 'allCaps'
        elif caps[0] == False and sum(caps) == len(word)-1:
            return 'upperInitial'
        elif 0 < sum(caps) < len(word):
            return 'mixedCaps'
        else:
            return 'noinfo'    
    
    @timeit
    def buildVocab(self, vocabSize=None, verbose=False, return_vocab_objects=False):
        """
        Builds the vocabulary based on the initial data file
        
        vocabSize(int, default: None-all words) - max number of words to use for vocabulary
                                                  (only used for training)
        verbose(boolean, default: False)        - print extra info
        """    	
        print ("----------------------------------------------------")
        print ("building vocabulary from TRAINING data...")

        flatData = [w for w in zip(*utils.flatten(self.train_sentences))]

        # remember these vocabs will have the <s>, </s>, and <unk> tags in there
        # sizes need to be interpreted "-3" - consider replacing...
        self.vocab = vocabulary.Vocabulary( flatData[0], size=vocabSize)
        self.posTags = vocabulary.Vocabulary( flatData[1])
        self.nerTags = vocabulary.Vocabulary( flatData[2])
        self.capitalTags = vocabulary.Vocabulary(flatData[3])

        if verbose:
            print ("vocabulary for words, posTags, nerTags built and stored in object")
            print ("vocab size =", vocabSize)
            print ("10 sampled words from vocabulary\n", list(self.vocab.wordset)[:10], "\n")
            print ("number of unique pos Tags in training =", self.posTags.size)
            print ("all posTags used\n", list(self.posTags.wordset), "\n")
            print ("number of unique NER tags in training =", self.nerTags.size)
            print ("all nerTags for prediction", list(self.nerTags.wordset), "\n")
            print ("number of unique capitalization tags in training =", self.capitalTags.size)
            print ('all capitalTags for prediction', list(self.capitalTags.wordset), "\n")

        if return_vocab_objects:
            return self.vocab, self.posTags, self.nerTags, self.capitalTags


    @timeit
    def formatWindowedData(self, sentences, windowLength=9, verbose=False):
        """
        Format the raw data by blocking it into context windows of a fixed length corresponding 
        to the single target NER tag of the central word.
        Make sure to call buildVocab first.
        
        sentences(list of lists of lists) - raw data from the CoNLL2003 dataset
        windowLength(int, default: 9)     - The length of the context window
                    NOTE - windowLength must be odd to have a central word. If itsn't, 1 will be added.
        verbose(boolean, default: False)  - print extra info
        
        Returns: 4 numpy arrays: vocabulary training data windowed and converted to IDs, 
                                 POS tags windowed and converted to IDs,
                                 Capitalization info windowed and converted into IDs,
                                 NER label tags converted to IDs
        """

        print ("----------------------------------------------------")
        print ("formatting sentences into input windows...")

        if windowLength % 2 == 0 or windowLength == 1:
            raise ValueError("window Length must be an odd number and greater than one.")
    
        pads = windowLength // 2

        # we have a list of lists (sentences) of lists ([word, posTag, nerTag])
        # parse through, pad each sentence with pads open and close tags, then convert to IDs
        vocabIDs = [ self.vocab.words_to_ids( ["<s>"] * pads + [word[0] for word in sent] + ["</s>"] * pads) \
                     for sent in sentences]
        posIDs = [ self.posTags.words_to_ids( ["<s>"] * pads + [word[1] for word in sent] + ["</s>"] * pads) \
                   for sent in sentences]
        capitalIDs = [self.capitalTags.words_to_ids(["<s>"]*pads + [word[3] for word in sent] + ["</s>"]*pads) \
                     for sent in sentences]
        nerIDs = [ self.nerTags.words_to_ids( ["<s>"] * pads + [word[2] for word in sent] + ["</s>"] * pads) \
                   for sent in sentences]
        
        if verbose: 
            print ("STEP 1/2 -- PADDING")
            print ("all sentences padded with {} pads to either end".format(pads))
            print ("vocab idx for first 5 sentences:\n", vocabIDs[:5], "\n")
            print ("pos idx for first 5 sentences:\n", posIDs[:5], "\n")
            print ("ner idx for first 5 sentences:\n", nerIDs[:5], "\n")
            print ("capitalization idx for first 5 sentences: \n", capitalIDs[:5], "\n")
            print ("number of sentences = {}".format(len(vocabIDs)), "\n")

        assert(len(vocabIDs) == len(posIDs) and len(posIDs) == len(nerIDs) == len(capitalIDs))

        if verbose: 
            print ("STEP 2/2 -- WINDOWING")

        # build the data to train on by sliding the window across each sentence
        # at this point, all 3 lists are the same size, so we can run through them all at once
        featsVocab, featsPOS, featsNER, featsCAPITAL = [], [], [], []
        for sentID in range( len(vocabIDs)):
            sent = vocabIDs[sentID]
            sentPOS = posIDs[sentID]
            sentNER = nerIDs[sentID]
            sentCAPITAL = capitalIDs[sentID]
            
            for ID in range( len(sent) - windowLength + 1):
                featsVocab.append( sent[ID:ID + windowLength])
                featsPOS.append( sentPOS[ID:ID + windowLength])
                featsCAPITAL.append(sentCAPITAL[ID:ID + windowLength])
                featsNER.append( sentNER[ID + windowLength // 2])
                
        if verbose:
            print ("sample windows:")
            for i in range(3):
                print ("Vocab for window {}".format(i))
                print (featsVocab[i])
                print (self.vocab.ids_to_words(featsVocab[i]))
                print ("PoS tags for window {}".format(i))
                print (featsPOS[i])
                print (self.posTags.ids_to_words(featsPOS[i]))
                print ("Capitalization tags for window {}".format(i))
                print (featsCAPITAL[i])
                print (self.capitalTags.ids_to_words(featsCAPITAL[i]))
                print ("NER tags for center word")
                print (featsNER[i])
                print (self.nerTags.ids_to_words([featsNER[i]]),"\n")
                
            print ("rows of vocab features = {}".format(len(featsVocab)))
            print ("rows of PoS features = {}".format(len(featsVocab)))
            print ("rows of Capitalization features = {}".format(len(featsCAPITAL)))
            print ("rows of NER features = {}".format(len(featsNER)))
            
            print ("numpy feature arrays are returned")

        assert(len(featsVocab) == len(featsVocab) == len(featsNER) == len(featsCAPITAL))
        return np.array(featsVocab), np.array(featsPOS), np.array(featsCAPITAL), np.array(featsNER)
    

## Load the Data

### 2.0

In [7]:


windowLength = 9
testNumSents = 5000

# Use training set to build vocab here
vocabData = conll2003Data(TRAIN_FILE)
vocabData.buildVocab( vocabSize=20000)

# Format training data
trainX, trainX_pos, trainX_capitals, trainY  = vocabData.formatWindowedData( vocabData.train_sentences, 
                                                  windowLength=windowLength,
                                                  verbose=False)

# read in dev data
devSents = vocabData.readFile( DEV_FILE)
devX, devX_pos, devX_capitals, devY = vocabData.formatWindowedData( devSents, 
                                              windowLength=windowLength,
                                              verbose=False)

# read in the test data
testSents = vocabData.readFile( TEST_FILE)
testX, testX_pos, testX_capitals, testY = vocabData.formatWindowedData( testSents, 
                                                windowLength=windowLength,
                                                verbose=False)

----------------------------------------------------
reading file from path ../data/conll2003/eng.train
'readFile'  1254.42 ms
----------------------------------------------------
building vocabulary from TRAINING data...
'buildVocab'  1048.01 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  1834.51 ms
----------------------------------------------------
reading file from path ../data/conll2003/eng.testa
'readFile'  301.05 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  423.75 ms
----------------------------------------------------
reading file from path ../data/conll2003/eng.testb
'readFile'  235.79 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  400.10 ms


### 2.1

In [8]:
# !
embed_dim = 50
# Load GloVe embedding matrix
embedding_matrix = construct_embedding_matrix(embed_dim)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.50d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 50))


### 2.2

In [8]:
# Get Y

# encoding 1-hot for ner targets
trainY_cat = to_categorical(trainY.astype('float32'))
devY_cat = to_categorical(devY.astype('float32'), num_classes=trainY_cat.shape[1])
testY_cat = to_categorical(testY.astype('float32'), num_classes=trainY_cat.shape[1])

trainY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), trainY_cat)), dtype=np.float)
devY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), devY_cat)), dtype=np.float)
testY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), testY_cat)), dtype=np.float)

### WIP here

In [77]:
# create dummy prediction labels
np.random.seed(0)
shuffle = np.random.permutation(np.arange(trainY_cat.shape[0]))
trainY_cat_pred = trainY_cat[shuffle]

In [88]:
trainY

array([5, 3, 7, ..., 3, 5, 3])

In [78]:
trainY_cat_pred

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [89]:
report1 = EvalDev_Report(y_true=trainY, raw_y_pred=trainY_cat_pred)

In [90]:
report1.f1

0.043738800928237814

In [91]:
report1.gold_cts

Counter({3: 169578,
         4: 11128,
         5: 10001,
         6: 8286,
         7: 4556,
         8: 37,
         9: 24,
         10: 11})

### 2.3

In [10]:
# !
# Get decoder Y -- 50 dim embedding of center word

train_decoderY = embedding_matrix[trainX[:,4]]
dev_decoderY = embedding_matrix[devX[:,4]]
test_decoderY = embedding_matrix[testX[:,4]]

### 2.4

In [11]:
# Get X pos tags

# encoding 1-hot for pos tags
trainX_pos_cat = to_categorical(trainX_pos.astype('float32'))
devX_pos_cat = to_categorical(devX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2]) 
testX_pos_cat = to_categorical(testX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2])

trainX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_pos_cat)), dtype=np.float)
devX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_pos_cat)), dtype=np.float)
testX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_pos_cat)), dtype=np.float)

### 2.5

In [12]:
# Get X capitlization 

# encoding 1-hot for capitalization info  ("allCaps", "upperInitial", "lowercase", "mixedCaps", "noinfo")
trainX_capitals_cat = to_categorical(trainX_capitals.astype('float32'))
devX_capitals_cat = to_categorical(devX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2]) 
testX_capitals_cat = to_categorical(testX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2])

trainX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_capitals_cat)), dtype=np.float)
devX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_capitals_cat)), dtype=np.float)
testX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_capitals_cat)), dtype=np.float)